In [1]:
%load_ext autoreload
%autoreload 2.0
%matplotlib inline

In [2]:
import os
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
import numpy as np
import sys
sys.path.insert(0, '../.')
from data.detection_dataset_loader import *
from scipy.spatial.transform import Rotation as R
from data.data_utils.data_reader import *
from data.data_utils.reader_utils import *
import math
import numpy.matlib as npm

from scipy.interpolate import RegularGridInterpolator, NearestNDInterpolator

/home/injy/mariam_workspace/env/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/injy/mariam_workspace/env/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/injy/mariam_workspace/env/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/injy/mariam_workspace/env/lib/python3.5/site-pack

In [3]:
def get_augmentation_parameters( training):
        if training:

                    if np.random.random_sample() >= 0.5:
                        image_translate_x = random.randint(-50, 50)
                    else:
                        image_translate_x = 0
                    if np.random.random_sample() >= 0.5:
                        image_translate_y = random.randint(-25, 25)
                    else:
                        image_translate_y = 0

                    if np.random.random_sample() >= 0.5:
                        translate_x = random.randint(-5, 5)
                    else:
                        translate_x = 0
                    if np.random.random_sample() >= 0.5:
                        translate_y = random.randint(-5, 5)
                    else:
                        translate_y = 0

                    if np.random.random_sample() >= 0.8:
                        translate_z = random.random() - 0.5
                    else:
                        translate_z = 0

                    if np.random.random_sample() >= 0.3:
                        ang = random.randint(-5, 5)
                    else:
                        ang = 0

                    r = R.from_rotvec(np.radians(ang) * np.array([0, 0, 1]))
                    rot = r.as_dcm()
                    rot = np.append(rot, np.array([[0,0,0]]), axis=0)
                    rot = np.append(rot, np.array([[0],[0],[0],[1]]), axis=1)

                    tr_x = translate_x
                    tr_y = translate_y
                    tr_z = translate_z
                    tr = np.array([[tr_x], [tr_y], [tr_z], [0]])
                    
                    translate_x = 0
                    translate_y = 0
                    translate_z = 0
                    
                    sc_x = 1
                    sc_y = 2
                    sc_z = 1

#                     if np.random.random_sample() >= 0.5:
#                        sc_x += ((random.random() * 2) - 1.) / 10.

#                     if np.random.random_sample() >= 0.5:
#                        sc_y += ((random.random() * 2) - 1.) / 10.


                    sc = np.array([[sc_x, 0, 0, 0], [0, sc_y, 0, 0], [0, 0, sc_z, 0], [0, 0, 0, 1]])
                    
                    fliplr = np.random.random_sample() >= 0.5

        else:
                    image_translate_x = 0
                    image_translate_y = 0

                    translate_x = 0
                    translate_y = 0
                    translate_z = 0
                    ang = 0

                    r = R.from_rotvec(np.radians(0) * np.array([0, 0, 1]))
                    rot = r.as_dcm()
                    rot = np.append(rot, np.array([[0,0,0]]), axis=0)
                    rot = np.append(rot, np.array([[0],[0],[0],[1]]), axis=1)

                    tr_x = 0
                    tr_y = 0
                    tr_z = 0
                    tr = np.array([[tr_x], [tr_y], [tr_z], [0]])

                    sc_x = 1
                    sc_y = 1
                    sc_z = 1
                    sc = np.array([[sc_x, 0, 0, 0], [0, sc_y, 0, 0], [0, 0, sc_z, 0], [0, 0, 0, 1]])
                    
                    fliplr = False

        return rot, tr, sc, image_translate_x, image_translate_y, ang, fliplr

In [4]:
training=True
base_path = '../../../Data/'
list_files = list(map(lambda x: x.split('.')[0], os.listdir(base_path+'/data_object_image_3/training/image_3/')))
# random.seed(0)
# random.shuffle(list_files)

# camera_paths = list(map(lambda x: base_path+'/data_object_image_3/training/image_3/' + x + '.png', list_files))
# lidar_paths = list(map(lambda x: base_path+'/data_object_velodyne/training/velodyne/' + x + '.bin', list_files))
label_paths = list(map(lambda x: base_path + '/data_object_label_2/training/label_2/' + x + '.txt', list_files))
calib_paths = list(map(lambda x: base_path + '/data_object_calib/training/calib/' + x + '.txt', list_files))
        
ln = int(len(list_files) * 0.5)
final_sample = len(list_files)

        
list_files = list_files[:len(list_files)]
list_label_paths = label_paths[:len(list_files)]
list_calib_paths = calib_paths[:len(list_files)]

In [5]:
def get_target2(labels, directions, anchors=np.array([3.9, 1.6, 1.5]), input_size=(800, 700), 
                output_size=(200, 175)):
    # ASSUMPTION: I will assume that the anchors contain a record for the height
    """
    - calculate the ratio = input size / final output size
    - divide each of the x, y and z by the ratio
    - subtract each of the x, y and z ....
    """

    ratio = input_size[0] // output_size[0]
    y_target = np.zeros((output_size[0], output_size[1], 2, 9), np.float32)
    old_angles = []
    angles = []
    angles_0 = []
    angles_1 = []
    for i in range(len(labels)):
        label_i = np.array(labels[i])

        x = int(label_i[0]/ratio)
        y = int(label_i[1]/ratio)

        if x >= output_size[0]:
            x = output_size[0] - 1
        if y >= output_size[1]:
            y = output_size[1] - 1

        if x < 0 or y < 0:
            continue

        label_i[0:2] = label_i[0:2] / (ratio*1.0)
        label_i[2] = label_i[2] / 36.
        old_angles.append(label_i[6])
        temp = label_i[6]
#         if temp < 0:
#             temp += np.pi

#         angle = temp * 57.2958
#         if (angle >= 0 and angle <= 45) or (angle >= 135 and angle <= 225) or (angle >= 315 and angle <= 360):
#             k = 0
#         else:
#             k = 1

        angle = temp * 57.2958
        if (angle >= 0 and angle <= 45) or (angle <= 0 and angle >= -45)\
            or (angle >= 135 and angle <= 180) or (angle <= -135 and angle >= -180):
            if angle >= 135:
                label_i[6] = label_i[6] - np.pi
            elif angle <= -135:
                label_i[6] = label_i[6] + np.pi
            k = 0
            angles_0.append(label_i[6])
        else:
            if angle < 0:
                label_i[6] = label_i[6] + np.pi
#             elif angle <= -90:
#                 label_i[6] = label_i[6] + np.pi
            angles_1.append(label_i[6])
            k = 1

#         if label_i[6] >= 3 * np.pi / 4:
#             label_i[6] = label_i[6] - np.pi
        
        label_i[6] = label_i[6] - k * (np.pi/2)
        
        angles.append(label_i[6])

        anchor = np.array([x+0.5, y+0.5, 1., anchors[0], anchors[1], anchors[2]])
    
        label_i[:3] = (label_i[:3] - anchor[:3]) / anchor[3:6]
        label_i[3:6] = np.log(label_i[3:6]/anchors)

        y_target[x, y, k, :7] = label_i
        y_target[x, y, k, 7:8] = [directions[i]]
        y_target[x, y, k, 8:9] = [1]
        
    return old_angles, angles, angles_0, angles_1
#     return y_target

In [49]:
def get_target3(labels, directions, anchors=np.array([3.9, 1.6, 1.5]), 
                input_size=(800, 700), output_size=(200, 175)):
    # ASSUMPTION: I will assume that the anchors contain a record for the height
    """
    - calculate the ratio = input size / final output size
    - divide each of the x, y and z by the ratio
    - subtract each of the x, y and z ....
    """

    ratio = input_size[0] // output_size[0]
    y_target = np.zeros((output_size[0], output_size[1], 9), np.float32)
    new_labels = []
    for i in range(len(labels)):
        label_i = np.array(labels[i])

        x = int(label_i[0]/ratio)
        y = int(label_i[1]/ratio)

        if x >= output_size[0]:
            x = output_size[0] - 1
        if y >= output_size[1]:
            y = output_size[1] - 1

        if x < 0 or y < 0:
            continue

        label_i[0:2] = label_i[0:2] / (ratio*1.0)
        label_i[2] = label_i[2] / 36.

        ang = label_i[6]
        label_i = np.append(label_i, [0])
        label_i[6:8] = [math.sin(ang), math.cos(ang)]
        
        anchor = np.array([x+0.5, y+0.5, 1., anchors[0], anchors[1], anchors[2]])
        label_i[:2] = (label_i[:2] - anchor[:2])
        label_i[3:6] = np.log(label_i[3:6])
        
        mins = np.array([-0.5, -0.5, 0, 0.7, 0.1, 0.1, -1.1, -1.1])
        maxs = np.array([0.5, 0.5, 1, 1.9, 0.75, 0.95, 1.1, 1.1])
        
        label_i = ((label_i - mins) / (maxs-mins)) * 2 - 1
        
        new_labels.append(label_i)

        y_target[x, y, :8] = label_i
        y_target[x, y, 8:9] = [1]
        
    return new_labels

In [54]:
new_labels = []
for i_c in range(len(list_files)):
    rot, tr, sc, image_translate_x, image_translate_y, ang, fliplr = get_augmentation_parameters(True)
    data_reader_obj = DataReader(None,
                                 list_calib_paths[i_c], 
                                 list_label_paths[i_c], 
                                 None, 
                                 rot, sc, tr, ang, image_translate_x, image_translate_y, get_actual_dims=False, fliplr=fliplr)
    _, label, directions = data_reader_obj.label_reader.read_label()
    labels2 = get_target3(label, directions,  anchors=np.array([3.9, 1.6, 1.5]))
    new_labels.extend(labels2)
    

In [55]:
new_labels = np.array(new_labels)

In [56]:
new_labels.shape

(26811, 8)

In [57]:
for i in range(8):
    print("i = ", i, ", min = ", np.min(new_labels[:, i]), ", max = ", np.max(new_labels[:, i]))

i =  0 , min =  -0.9998963033733617 , max =  0.9999579412402682
i =  1 , min =  -0.9998331144931285 , max =  0.9998722380235563
i =  2 , min =  -0.9157024847994941 , max =  0.9997140473069632
i =  3 , min =  -0.8601640936193176 , max =  0.9960330998792206
i =  4 , min =  -0.9045284233649108 , max =  0.8859994087880771
i =  5 , min =  -0.9269923237496377 , max =  0.6727769793325384
i =  6 , min =  -0.9090907402334414 , max =  0.9090907402334416
i =  7 , min =  -0.9090909090909091 , max =  0.909090909090909


In [53]:
for i in range(8):
    print("i = ", i, ", min = ", np.min(new_labels[:, i]), ", max = ", np.max(new_labels[:, i]))

i =  0 , min =  -0.9999763375340649 , max =  0.9999169108402839
i =  1 , min =  -0.9999584360982681 , max =  0.9999856834787977
i =  2 , min =  -0.9157024847994941 , max =  0.9995024961871322
i =  3 , min =  -0.8601640936193176 , max =  0.9960330998792206
i =  4 , min =  -0.9045284233649108 , max =  0.8859994087880771
i =  5 , min =  -0.9269923237496377 , max =  0.9017848474750372
i =  6 , min =  -0.9090906208471223 , max =  0.9090906208471221
i =  7 , min =  -0.9090897561159449 , max =  0.909090909090909


In [7]:
list_angles = np.array(list_angles)
list_original_angles = np.array(list_original_angles)
list_angles_0 = np.array(list_angles_0)
list_angles_1 = np.array(list_angles_1)

In [8]:
np.max(list_angles), np.min(list_angles)

(0.7815926535897932, -0.7815926535897932)

In [9]:
np.max(list_angles_0), np.min(list_angles_0)

(0.7815926535897932, -0.7815926535897932)

In [10]:
np.max(list_angles_1), np.min(list_angles_1)

(2.351592653589793, 0.79)

In [11]:
np.max(list_original_angles), np.min(list_original_angles)

(3.14, 0.0)

In [19]:
np.max(list_angles), np.min(list_angles)

(0.7800000000000002, -0.7815926535897932)

In [20]:
np.max(list_original_angles), np.min(list_original_angles)

(3.14, 0.0)

In [23]:
np.pi, np.pi/2, np.pi/4

(3.141592653589793, 1.5707963267948966, 0.7853981633974483)

In [7]:
i_c = 3
rot, tr, sc, image_translate_x, image_translate_y, ang, fliplr = get_augmentation_parameters(False)
data_reader_obj = DataReader(None,
                                 list_calib_paths[i_c], 
                                 list_label_paths[i_c], 
                                 None, 
                                 rot, sc, tr, ang, image_translate_x, image_translate_y, get_actual_dims=False, fliplr=fliplr)
_, label, directions = data_reader_obj.label_reader.read_label()
label2 = get_target(label, directions,  anchors=np.array([3.9, 1.6, 1.5]))

In [9]:
len(label), label

(1, array([[278.3097315 , 178.11101949,  26.43074065,   4.26      ,
           1.74      ,   1.47      ,   3.09      ]]))

In [10]:
np.where(label2[:, :, :, 8] >= 0.5)

(array([69]), array([44]), array([0]))

In [11]:
label2[69, 44, 0, :]

array([ 0.01985458,  0.0173468 , -0.22615433,  0.08829261,  0.08388148,
       -0.02020271, -0.05159265,  1.        ,  1.        ], dtype=float32)

In [41]:
np.log(1.74/1.6)

0.08388148398070203

In [56]:
np.log(2/1.6), 2/1.6, np.log(2.5/1.6), 2.5/1.6

(0.22314355131420976, 1.25, 0.44628710262841953, 1.5625)

In [49]:
(abs(0.22314355131420976-0.08388148398070203)**2) * 0.5

0.009696961699001222

In [40]:
np.exp(0.08829261) * 3.9

4.26000001215941

In [42]:
temp = 277
# temp = 278.3097315
((temp/4 - temp//4) - 0.5)/3.9, temp//4

(-0.06410256410256411, 69)

In [45]:
abs(-0.06410256410256411 - 0.01985458) - 0.5

-0.4160428558974359

In [ ]:
0.06410256410256411

In [25]:
(0.01985458*3.9 + 69.5) * 4

278.309731448

In [54]:
(0.01985458*3.9 + .5), (0.01985458*3.9 + .5) * 4

(0.577432862, 2.309731448)

In [16]:
0.01985458*3.9 + 0.5, 0.0173468*1.6 + 0.5, -0.22615433*1.5 + 1

(0.577432862, 0.52775488, 0.6607685050000001)

In [17]:
278.3097315 / 4, 278.3097315//4, 178.11101949/4, 178.11101949//4, 26.43074065/40, 26.43074065//40

(69.577432875, 69.0, 44.5277548725, 44.0, 0.66076851625, 0.0)

In [20]:
69.577432875 * 4

278.3097315

In [21]:
(0.3097315 - 0.5) / 3.9

-0.048786794871794874

In [23]:
(0.577432875 - 0.5) / 3.9

0.01985458333333332